# **Simulating F1-ATPase Rotation Dynamics**
<hr>

## **Introduction**

Contained within this notebook is the complete implementation of **Langevin dynamics using stochastic methods**. <br>
F1-ATPase has the ability to convert chemical energy into mechanical energy. Due to this, F1-ATPase has important implications <br>
for fields such as biomechanical engineering. <br>

This notebook will successfully do the following:<br>
1) **Simulate rotational dynamics of F1-ATPase alongside stochastic restrains**.
2) **Prove generating sufficient and meaningful data for AI models is possible**.

<br>
<hr>

## **Theory**
The simulation models the F1-ATPase motor using a hybrid Langevin Gillespie approach. 

1) Gillespie Algorithm: Determines the motor's current chemical state
2) Langevin Dynamics: Pulls the beads current angle toward an equilibrium point, defined by the current state

### **1. State Transitions (Gillespie)**


The Gillespie algorithm helps model the motors jump between chemical states. In order to achieve this, at every time step ($\Delta t$), it will determine if a transition occurs. If a transition occurs it will determine what new state the system belongs to.

The process provides the necessary information in order to calculate the new target angle:
* The `transition_matrix` stores the probabilities of these jump, from one state to another
* The algorithm uses these rates to determine if a state change occurs each step ($\Delta t$)
* Through this method two variables are returned
  * A `cycle_count` variable tracks full rotations (forward and backward) in order to ensure the angle updates properly
  * A `new_state` variable tracks the current state of the system
* Using these variables allows us to finally update our angle toward the current equilibrium point 


$$
\theta_{\rm target} = \theta_{\rm state} + cycle_{count} \cdot \frac{2\pi}{3}
$$

##### **Corresponding Code**
```C++
   double LangevinGillespie::update_theta(int state, int cycle_count) const {              
       return theta_states->at(state) + static_cast<double>(cycle_count) * 2.0             
                                      * std::numbers::pi / 3.0;                           
   }                                                                                       


```

### **2. Angle Updates (Langevin)**

Once the Gillespie step determines our new target angle ($\theta_{\rm target}$), Langevin dynamics can simulate the continuous physical rotation of the bead being pulled toward the equilibrium point.

The bead acts as if it is in a well, trying to reach the equilibrium point. Despite its efforts, external factors such as thermal energy ($\text{k}_{\rm B} T$) cause random fluctuations that kick the bead around. From this summary the simulation will have to account for two forces:
* **Drift (Deterministic)**: Acts similar to a spring, in the sense that the force pulls the beads current angle ($\theta$) toward the target angle ($\theta_{\rm target}$). Its strength is determined by the elastic constant $\kappa$
* **Diffusion (Stochastic)**: Represents those random "kicks" from thermal noise (Brownian motion), in which relates to thermal energy ($\kappa_{\rm B} T$)

Although three angle update methods are implemented (Heun, Euler-Maruyama, and Probabilistic), Heun will be chosen for its exceptional accuracy. 

$$
\theta(t+\Delta t) = \theta(t)
+ \frac{\Delta t}{2} \left( \text{drift}_{\rm term} + \text{drift}_{\rm predict} \right)
+ \sqrt{\Delta t} \, \text{diffusion}_{\rm term} \, \eta
$$

##### **Corresponding Code**
```C++
    double LangevinGillespie::heun_1d(                                                     
        double current_angle,                                                              
        double theta_target,                                                               
        std::mt19937& local_rng                                                            
        ) const {                                                                          
        double drift_term = drift(current_angle, theta_target);                            
        double diffusion_term = diffusion();                                               
        double eta = normal_dist(local_rng);                                               
                                                                                           
        // Predictor step                                                                  
        double sqrt_dt = std::sqrt(dt.value());                                            
        double y_predict = current_angle + dt.value() * drift_term + sqrt_dt               
        * diffusion_term * eta;                                                            
                                                                                           
        // Corrector step                                                                  
        double drift_predict = drift(y_predict, theta_target);                             
        return current_angle + (dt.value() / 2.0) * (drift_term + drift_predict)           
                             + sqrt_dt * diffusion_term * eta;                             
   }                                                                                       
```
<hr>


## **Design Decisions: Why C++ / CUDA?**
Before we dive into implementation lets consider our second goal of this project: prove generating sufficient and meaningful data for an AI model is possible. <br>
Generating a large dataset for AI models requires lots of data points. This means we will have to run this simulation multiple times.

### **The Problem: Python is too slow!**
The first implementation of this project started within Python. This was wonderful for prototyping and validating physics, but it was far too slow for data generation.

Some primary bottlenecks within Python include its Global Interpreter Lock (GIL), which prevents true parallelism, alongside being an interpreted language. These issues made scaling simulations efficiently unreachable, as shown with the legacy benchmark. (Results below)

### **The Solution: C++, Cuda, and Pybind11**
We are able to take full advantage of Python's ecosystem while also achieving blazingly fast speeds, this is why I used Pybind11.

Pybind11 allows us to write Python Code, while directly accessing C++/CUDA code. The code has also been rewritten to support massive amounts of parallelization, and bypassing the GIL. 

### **The Results: A 314 times speed up**
The results of this optimization are absolutely incredible. The chart below compares the time taken to run 10,000 simulations (100k steps each) using different implementations.
The original "Legacy" (Purely Python) code took around 88.59 minutes (5315.62s) to complete. The final CUDA version finished the same job within 16.88 seconds. 

This is a **314.88 times** increase in computation speed. This makes our goal of generating AI datasets not only possible, but trivial.  

![Time Comparison for 10k simulations, with 100k steps](analysis/results/time_comparison_10k.png)

<hr>

## **Simulation Setup**

Since I have established the what (Theory) and why (Performance), I can finally show you the how. Before running any simulations we must first import our libraries and parameters.

### **Imports**
First lets import all needed libraries, in which includes our `f1sim` module, and `compute_transition_matrix` helper

In [1]:
from bin.f1sim import LangevinGillespie
from src.utils.compute_transition_matrix import compute_transition_matrix

# -=-=-=-=-=-=-=-=-= Helper Imports -=-=-=-=-=-=-=-=-=
import numpy as np
import math


## Simulation configuration
Next, we will create an instance of our `LangevinGillespie` class and define our parameters

In [2]:
# 1. Create our class instance
LG = LangevinGillespie()

# 2. Simulation Parameters
LG.steps = 2000
LG.dt = 1e-6
LG.method = "heun"

# 3. Mechanical / Thermal Setup 
LG.kappa = 56
LG.kBT = 4.14
LG.gammaB = LG.computeGammaB(a=20, r=19, eta=1e-9)

#  4. Multi State Setup 
LG.theta_states = np.array([3, 36, 72, 116]) * math.pi / 180  # Deg → Rad
LG.initial_state = 0 # Start at 3 degrees, This step is optional

#  5. Transition rate matrix 
LG.transition_matrix = compute_transition_matrix(LG)

## **Data Generation**

Now that LG is fully configured, we can start generating data. I will show you three different simulation methods, along with their strengths
1. simulate(optional int: seed): Generates a single trajectory, no thread overhead meaning better times for very small datasets
2. simulate_multithreaded(unsigned int: nSim,unsigned int: numThreads, optional int: seed): Distributes the number of simulations on the CPU. Ideal for fast simulations without the use of a dedicated GPU. 
3. simulate_multithreaded_cuda(unsigned_int: nSim, optional int: seed): Runs all nSim simulations in parallel on the GPU. Blazingly fast, but requires the a Nvidia GPU

### **Running a single simulation**
First, lets run a single simulation, and save its trajectory for analysis

In [3]:
import time

SEED_VAL = 0
last = time.time()

angles, states, thetas = LG.simulate(seed=SEED_VAL)

now = time.time()
print(f"Simulation complete in {now - last:.4f} seconds")

Simulation complete in 0.0002 seconds


### **Generating our Datasets**

We have two high performance parallel methods (`simulate_multithreaded` and `simulate_multithreaded_cuda`).

As a final demonstration, we will run a benchmark of 100,000 simulations. This will show the practical performance difference between the CPU and GPU. The resulting data will be used in the "Analysis" section

In [4]:
N_SIMS = 100_000
CPU_THREAD_NUMBER = 32

# -=-=-=-=-=-=-=-=-= 1. CPU Multithreaded -=-=-=-=-=-=-=-=-=
print(f"Running {N_SIMS} simulations on {CPU_THREAD_NUMBER} CPU threads...")
start_time_cpu = time.time()

combined_bead_positions_cpu, combined_states_cpu, combined_target_thetas_cpu = (
    LG.simulate_multithreaded(N_SIMS, CPU_THREAD_NUMBER, seed=SEED_VAL)
)

end_time_cpu = time.time()
cpu_time = end_time_cpu - start_time_cpu
print(f"  CPU simulation complete in: {cpu_time:.4f} seconds")


# -=-=-=-=-=-=-=-=-= 2. GPU (CUDA) -=-=-=-=-=-=-=-=-=
print(f"\nRunning {N_SIMS} simulations on the GPU...")
start_time_gpu = time.time()

combined_bead_positions_gpu, combined_states_gpu, combined_target_thetas_gpu = (
    LG.simulate_multithreaded_cuda(N_SIMS, seed=SEED_VAL)
)

end_time_gpu = time.time()
gpu_time = end_time_gpu - start_time_gpu
print(f"  GPU simulation complete in: {gpu_time:.4f} seconds")


# -=-=-=-=-=-=-=-=-= 3. Results -=-=-=-=-=-=-=-=-=
print(f"\n Speedup (GPU vs CPU): {cpu_time / gpu_time:.2f}x")

Running 100000 simulations on 32 CPU threads...
  CPU simulation complete in: 4.4781 seconds

Running 100000 simulations on the GPU...
  GPU simulation complete in: 2.8922 seconds

 Speedup (GPU vs CPU): 1.55x


<hr>

## **Analysis**

Now that the datasets are generated, lets analyze the results. We can start by visualizing the trajectory with `simulate()`

`Note`: Our simulation uses radians for its input and outputs, all graph data has been converted to degrees

### **Visualizing a Single Trajectory**
The plot below shows, that the model is indeed behaving as expected.
1. Top Plot: The **Bead Angle** is being pulled toward the **Target Angle**. You can see the random kicks caused by diffusion, as the angle fluctuates around the target.
2. Bottom Plot: The **State** shows each discrete jump, from the Gillespie algorithm.
3. Colored Background: The background color, represents the current state. This changes any time our state changes
   1. (Blue: State 0, Orange: State 1, Green, State 2, Red: State 3)

![Single Simulation Trajectory](analysis/results/single_simulation_results_seed0.png)

## **Single Trajectory PDF**
The histogram is color coded to match the trajectory plot above, with each color representing a probability distribution of the bead while it remained in that state. Each segment is individually normalized, so the area under each colored PDF sums to 1, reflecting the probability distribution of the bead in that state during a single trajectory.

* Tall, narrow peaks correspond to states where the bead passes quickly, with angular positions tightly clustered within that segment. The high PDF values reflect this concentration over a small range of angles.
* Wide, shorter peaks correspond to states where the bead dwells longer, but its positions fluctuate over a broader range. The PDF values are lower because the probability is spread across more angles.


![Single Simulation PDF](analysis/results/probability_density.png)


<hr>

### Thread Support
The simulation allows simulations to happen through the CPU via C++ and GPU via CUDA.
This is to bypass pythons GIL and interpreter.

In [5]:
MAX_THREAD_COUNT = 32
SIMULATION_COUNT = 32
combined_bead_positions, combined_states, combined_target_thetas = (
    LG.simulate_multithreaded(SIMULATION_COUNT, MAX_THREAD_COUNT)
)

![Multithreaded Simulation Trajectory](analysis/results/multithreaded_simulation_results.png)

<hr>

## Cuda Support

In [6]:
SIMULATION_COUNT_CUDA = 64
combined_bead_positions, combined_states, combined_target_thetas = (
    LG.simulate_multithreaded_cuda(SIMULATION_COUNT_CUDA)
)

### CUDA Simulation Trajectories
![Cuda Simulation Trajectory](analysis/results/cuda_simulation_results.png)